In [1]:
import requests, six
import pandas as pd

In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
import lxml.html as lh

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # The Link to get information

### Inspect webpage with Inspect Element (Q) of Firefox browser
![Inspect Element (Q)](firefoxinspect.JPG)
### Examine the Toronto Postal code, we see:
* Titles of columns locate in "tr/th...th/tr" frame
* The items of cells locate in "tr/td...td/tr" frame 

In [5]:
Table = requests.get(url) # Store the contents of the table under Table
Content = lh.fromstring(Table.content) # Parse data that locate between <tr>..</tr> of HTML
tr_items = Content.xpath('//tr')

In [6]:
#Check the length of the first 10 rows
[len(Row) for Row in tr_items[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [7]:
tr_items = Content.xpath('//tr') # Create empty list
col=[]
i=0 # Fill the first row with the "headers" or name of columns.
for t in tr_items[0]:
    i+=1
    name=t.text_content()
    print ('%d:'"%s"''%(i,name))
    col.append((name,[]))

1:Postal Code

2:Borough

3:Neighborhood



In [8]:
# Start fill data in the second row

for j in range(1,len(tr_items)):
    
    # R is j'th row
    R=tr_items[j]
    
    # Row size (R) is 3, if not the content is not from table  
    if len(R) != 3:
    
     break    
    
    # i is the index of column
    i=0
    
    #Iterate through each item of the row
    for t in R.iterchildren():
        data=t.text_content() 
        
        #Check if row is empty, and format to integer
        if i>0:        
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column, start from second row or [1]
        col[i][1].append(data)
        
        i+=1 # Go to next row

In [9]:
# Inspect the content in each column or how many items in each column
[len(C) for (title,C) in col]

[181, 181, 181]

In [10]:
Tb_dict={title:column for (title,column) in col}
df=pd.DataFrame(Tb_dict)

In [11]:
df.head(5)

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [12]:
df.tail(5)

,Postal Code\n,Borough\n,Neighborhood\n
176,M6Z\n,Not assigned\n,Not assigned\n
177,M7Z\n,Not assigned\n,Not assigned\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z\n,Not assigned\n,Not assigned\n
180,\n,Canadian postal codes\n,\n


In [13]:
print(df.dtypes)

Postal Code\n     object
Borough\n         object
Neighborhood\n    object
dtype: object


After inspect the content of the table, We need to 
* Remove the "\n" 
* Change the names of columns 
* Remove rows with "not assigned" in "Borough" column 
* Check the "Neighborhood" column, if the row has "not assigned" we need to replace with the same content of "Borough" column.

In [14]:
# Rename the column name, drop all '\n', and delete the last row "Canadian Postal codes"
df.columns=["PostalCode", "Borough","Neighborhood"]
df = df.replace('\n','', regex=True)
df.drop(df.index[180], inplace = True)

In [15]:
df['Borough'].value_counts()


Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [16]:
# Delete all rows having " Not assigned" in the "Borough" column

df.drop(df[df['Borough'] == "Not assigned"].index, inplace = True) 

In [17]:
# Check if the "Not assigned" value in the Neighborhood column.
count = df[df['Neighborhood']=="Not assigned"]
print(count)

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []


In [18]:
df.reset_index(drop=True, inplace=True)
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [19]:
print(df.shape)

(103, 3)


In [20]:
df.sort_values("PostalCode", axis = 0, ascending = True,inplace = True) 

In [21]:

geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [22]:
geo_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [23]:
geo_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [24]:
uni_df = pd.merge(df, geo_df, on='PostalCode')

In [25]:
uni_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
